<a href="https://colab.research.google.com/github/Intruder9211/python/blob/main/TextRetrivel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install git+https://github.com/openai/CLIP.git
!pip install torch torchvision
!pip install pyttsx3
!pip install opencv-python matplotlib nltk


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-fostda1_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-fostda1_
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.8 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=0e57eeef50a9f30ac567cf9a7756e9e354c3381e3da6f78cd3e1b5763e2adada
  Stored in directory: /tmp/pip-ephem-wheel-cache-se1ag7d2/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [2]:
import os

# Define the main project directory
project_dir = "/content/image-captioning"

# Create the main project folder
os.makedirs(project_dir, exist_ok=True)

# Create subfolders for datasets, images, and saved models
os.makedirs(os.path.join(project_dir, "datasets"), exist_ok=True)
os.makedirs(os.path.join(project_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(project_dir, "saved_models"), exist_ok=True)

# Create empty placeholder files
for file_name in ["main.py", "requirements.txt", "README.md"]:
    file_path = os.path.join(project_dir, file_name)
    with open(file_path, "w") as f:
        f.write("")  # Create an empty file

print(f"Folder structure created at: {project_dir}")


Folder structure created at: /content/image-captioning


In [3]:
!ls -R /content/image-captioning



/content/image-captioning:
datasets  images  main.py  README.md  requirements.txt	saved_models

/content/image-captioning/datasets:

/content/image-captioning/images:

/content/image-captioning/saved_models:


In [4]:
from PIL import Image
import clip
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)

# Load and preprocess image
image_path = "/content/image-captioning/images/deadpool.jpg"  # Add your image path
image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)


100%|███████████████████████████████████████| 338M/338M [00:15<00:00, 22.7MiB/s]


In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")

# Generate captions
def generate_caption(image_features):
    input_text = "A picture of"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    outputs = gpt2_model.generate(
        input_ids=input_ids,
        max_length=50,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Encode image with CLIP and generate caption
with torch.no_grad():
    image_features = clip_model.encode_image(image)

caption = generate_caption(image_features)
print("Generated Caption:", caption)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Caption: A picture of a man with his head covered in blood and his face covered with a blood-soaked mask.

The man, who was not identified, was taken to a local hospital, where he was pronounced dead. He was wearing a


In [7]:
torch.save(gpt2_model.state_dict(), "/content/image-captioning/saved_models/gpt2_model.pth")



In [8]:
from google.colab import files
files.download("/content/image-captioning/saved_models/gpt2_model.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>